## ***Testing Single Model and Generating Confusion Matrix***

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the path to the saved model
saved_model_path = '/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/INCEPTv3_V_ACC.h5'

# Load the saved model
loaded_model = load_model(saved_model_path)

# Set your data directory
data_directory = '/content/drive/MyDrive/Thyroid Nodule Classification/Imaging_Data'

# Specify the class names here (in the same order as used during training)
class_names = sorted(os.listdir(os.path.join(data_directory, 'testing_data')))
print(class_names)
# Data generator for testing
test_data_generator = ImageDataGenerator(rescale=1./255)
test_data_flow = test_data_generator.flow_from_directory(
    os.path.join(data_directory, 'testing_data'),
    target_size=(299, 299),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# Make predictions using the loaded model
y_true = test_data_flow.classes

y_pred = loaded_model.predict(test_data_flow, verbose=1)

#Print Predicted and Real Class
for i in range(len(y_pred)):
    predicted_class = np.argmax(y_pred[i])

    if predicted_class == 0:
      Pred_Class='Blocked'
    elif predicted_class == 1:
      Pred_Class='Cold'
    elif predicted_class == 2:
      Pred_Class='Functional'
    elif predicted_class == 3:
      Pred_Class='MNG'
    elif predicted_class == 4:
      Pred_Class='Normal'
    else: Pred_Class='Unknown'

    true_class = y_true[i]

    if true_class == 0:
      Actutal_Class='Blocked'
    elif true_class == 1:
      Actutal_Class='Cold'
    elif true_class == 2:
      Actutal_Class='Functional'
    elif true_class == 3:
      Actutal_Class='MNG'
    elif true_class == 4:
      Actutal_Class='Normal'
    else: Actutal_Class='Unknown'
    #print(f"Example {test_data_flow.filepaths[i]} - Predicted: {Pred_Class}, True: {Actutal_Class}")

# Calculate accuracy, precision, recall, F1-score, and confusion matrix
classification_rep = classification_report(y_true, np.argmax(y_pred, axis=1), target_names=class_names)
confusion_mtx = confusion_matrix(y_true, np.argmax(y_pred, axis=1))

print("Classification Report:")
print(classification_rep)

print("Confusion Matrix:")
print(confusion_mtx)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Blocked', 'Cold', 'Functional', 'MNG', 'Normal']
Found 95 images belonging to 5 classes.
95/95 [==============================] - 60s 627ms/step
Classification Report:
              precision    recall  f1-score   support

     Blocked       0.86      1.00      0.92         6
        Cold       0.78      0.78      0.78        36
  Functional       0.78      0.88      0.82         8
         MNG       0.80      0.67      0.73        18
      Normal       0.89      0.93      0.91        27

    accuracy                           0.82        95
   macro avg       0.82      0.85      0.83        95
weighted avg       0.82      0.82      0.82        95

Confusion Matrix:
[[ 6  0  0  0  0]
 [ 1 28  1  3  3]
 [ 0  1  7  0  0]
 [ 0  5  1 12  0]
 [ 0  2  0  0 25]]


## ***Testing on All Trained Models & Generating Excel file for Comparison***

In [16]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from openpyxl import Workbook
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set your data directory
data_directory = '/content/drive/MyDrive'

# Specify the class names here (in the same order as used during training)
class_names = sorted(os.listdir(os.path.join(data_directory, 'testing_data')))

# Data generator for testing
test_data_generator = ImageDataGenerator(rescale=1./255)
test_data_flow = test_data_generator.flow_from_directory(
    os.path.join(data_directory, 'testing_data'),
    target_size=(299, 299),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# Load and evaluate models from a folder
models_folder = '/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models'
model_files = [f for f in os.listdir(models_folder) if f.endswith('.h5')]

results = []

for model_file in model_files:
    model_path = os.path.join(models_folder, model_file)
    print(model_path)
    loaded_model = tf.keras.models.load_model(model_path)

    # Make predictions using the loaded model
    y_true = test_data_flow.classes
    y_pred = loaded_model.predict(test_data_flow, verbose=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Calculate metrics
    classification_rep = classification_report(y_true, y_pred_classes, target_names=class_names,
                                               output_dict=True, zero_division='warn')  # Handle undefined metrics
    confusion_mtx = confusion_matrix(y_true, y_pred_classes)
    #print(confusion_mtx)
    f1 = f1_score(y_true, y_pred_classes, average='weighted', zero_division='warn')  # Handle undefined metrics



    results.append({
        'Model': model_file,
        'Classification_Report': classification_rep,
        'Confusion_Matrix': confusion_mtx,
        'Weighted_F1_Score': f1
    })

# Create a comparison table for all parameters
comparison_table = []
for result in results:
    comparison_row = {
        'Model': result['Model'],
        #'Confusion_Matrix' : result['Confusion_Matrix'],
        'Weighted_F1_Score': result['Weighted_F1_Score']

    }
    for class_name in class_names:
        comparison_row[f'{class_name} Precision'] = result['Classification_Report'][class_name]['precision']
        comparison_row[f'{class_name} Recall'] = result['Classification_Report'][class_name]['recall']
        comparison_row[f'{class_name} F1-Score'] = result['Classification_Report'][class_name]['f1-score']
    comparison_table.append(comparison_row)

# Save results and comparison to an Excel file
#excel_file = r'/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/models_evaluation_results(With_Confusion_Matrix).xlsx'
excel_file = r'/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/models_evaluation_results.xlsx'
workbook = Workbook()
comparison_sheet = workbook.active
comparison_sheet.title = 'Comparison'

import pandas as pd


# Create a DataFrame from the data
df = pd.DataFrame(comparison_table)

# Save the DataFrame to an Excel file
#excel_file_path = 'output_data.xlsx'
df.to_excel(excel_file, index=False)

print(f"Data has been saved to '{excel_file}'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 95 images belonging to 5 classes.
/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/RESNET50_ACC.h5
95/95 [==============================] - 2s 13ms/step
/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/RESNET101_V_ACC.h5
95/95 [==============================] - 4s 21ms/step
/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/RESNET101_ACC.h5
95/95 [==============================] - 4s 26ms/step
/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/RESNET50_V_ACC.h5
95/95 [==============================] - 2s 13ms/step
/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/DENSE_V_ACC.h5
95/95 [==============================] - 4s 14ms/step
/content/drive/MyDrive/Thyroid Nodule Classification/Trained_Models/DENSE_ACC.h5
95/95 [==============================] - 4s 19ms/step
/